In [1]:
import pandas as pd
import pickle
from cryptography.fernet import Fernet

In [2]:
df = pd.read_csv('original_data/relatorio_votos.csv')

In [3]:
df.keys()

Index(['Código', 'Data de criação', 'Autor', 'Título', 'Descrição',
       'Categoria', 'Resultado da priorização', 'Votos',
       'Resultado da votação', 'Viabilidade', 'Compromisso', 'Justificativa',
       'Data do voto', 'Horário', 'documento', 'subprefeitura'],
      dtype='object')

In [4]:
def get_fernet_crypto():
    
    with open('original_data/fernet_key.pi', 'rb') as f:
        key = pickle.load(f)
        
    return Fernet(key)

In [5]:
def encrypt(item, crypto=None):
    
    if crypto is None:
        crypto = get_fernet_crypto()
    
    bytes_item = str(item).encode('utf-8')
    
    return crypto.encrypt(bytes_item)

In [6]:
def decrypt(item, crypto=None):
    
    if crypto is None:
        crypto = get_fernet_crypto()
    
    return crypto.decrypt(item).decode('utf-8')

In [7]:
decrypt(encrypt('Henrique Pougy'))

'Henrique Pougy'

In [8]:
df['Autor'].fillna('Vazio', inplace = True)
df['Autor'] = df['Autor'].apply(encrypt)
df['documento'].fillna('Vazio', inplace = True)
df['documento'] = df['documento'].apply(encrypt)

In [9]:
def drop_cols_sem_info(df):
    
    df = df.copy()
    
    remover = []
    for col in df.columns:
        val_unicos = df[col].unique()
        if len(val_unicos) == 1:
            print(f'A coluna {col} possui como valor unico "{val_unicos[0]}"')
            remover.append(col)
    df.drop(remover, axis=1, inplace = True)
    
    return df

In [10]:
df = drop_cols_sem_info(df)

A coluna Resultado da priorização possui como valor unico "Priorizada"
A coluna Viabilidade possui como valor unico "Etapa orçamentária futura"
A coluna Compromisso possui como valor unico "Etapa orçamentária futura"
A coluna Justificativa possui como valor unico "Etapa orçamentária futura"


In [11]:
df.head()

,Código,Data de criação,Autor,Título,Descrição,Categoria,Votos,Resultado da votação,Data do voto,Horário,documento,subprefeitura
0,335,17/04/2021,b'gAAAAABhACQW-_be27MbRIPQgCqqQRBRA0ENPZ0SIX3D...,Implantação do Parque Cabeceira do Cabore,Implantação do Parque Cabeceira do Cabore Pl 7...,Meio Ambiente,491,Eleita,25/06/21,00:05,b'gAAAAABhACQYEW7IvekWboY98mYagGw2x9vB9tg4QB0G...,Campo Limpo
1,81,13/04/2021,b'gAAAAABhACQWZx10J1RiKcNC-FDx6r7NiHh2sbVzLCkr...,Implantação de UBS na região do parque industr...,Implantação de uma unidade básica de saúde na ...,Saúde,136,Não eleita,25/06/21,00:05,b'gAAAAABhACQYowvYMUSUkEGsr0xPVN9zQiwKCsT9ARLp...,Lapa
2,398,18/04/2021,b'gAAAAABhACQWMLqrsVo93R86CRwDaU_bOpmdmqXVvNXK...,"Implementação dos parques Horto do Ipê, Capadó...",Conforme o Decreto 53.965/2013 que declara a á...,Meio Ambiente,1072,Eleita,25/06/21,00:05,b'gAAAAABhACQYAkJa5gC2evmp8NLrkmI8mzp9QjKSI2W_...,Campo Limpo
3,429,19/04/2021,b'gAAAAABhACQWN1PImxdf6xfmGAqUVrtpyHA-JbzXQna4...,Construção de um Hospital no distrito do Campo...,Construção de um Hospital no distrito do Campo...,Saúde,552,Eleita,25/06/21,00:07,b'gAAAAABhACQY62p3DPPgdJ2alkpYMPJHki-vPAxTeuLS...,Campo Limpo
4,3148,10/05/2021,b'gAAAAABhACQWvnjjiuzSpY5AnkQUcAObwm1Bc8TfHG1A...,Pavimentação e Drenagem na Rua Servidão V,Pavimentação e Drenagem na Rua Servidão V (Rua...,Zeladoria Urbana e melhorias de bairro,78,Não eleita,25/06/21,00:08,b'gAAAAABhACQYKa-WkWUB0DvcihY5YDtkbecmxJMLc_g-...,São Mateus


In [13]:
df.to_csv('microdados_votacao_orcamento_anonimizados.csv', sep =';', index = False, encoding = 'utf-8')

In [14]:
df = pd.read_csv('microdados_votacao_orcamento_anonimizados.csv', sep = ';')

In [16]:
df.dtypes

Código                   int64
Data de criação         object
Autor                   object
Título                  object
Descrição               object
Categoria               object
Votos                    int64
Resultado da votação    object
Data do voto            object
Horário                 object
documento               object
subprefeitura           object
dtype: object